1. predict the average case for each city
2. predict using the average for each city AND each week of year

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
#load google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [6]:
X_train = pd.read_csv("dengue_features_train.csv")
y_train = pd.read_csv("dengue_labels_train.csv")

In [7]:
df = X_train.merge(y_train, on=['city', 'year', 'weekofyear'])

In [8]:
df.columns

Index(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'total_cases'],
      dtype='object')

In [9]:
df.groupby(["city","weekofyear"])["total_cases"].mean()

city  weekofyear
iq    1              9.300000
      2             13.800000
      3             10.800000
      4             15.800000
      5             14.800000
                      ...    
sj    49            47.055556
      50            40.055556
      51            33.555556
      52            38.000000
      53            33.000000
Name: total_cases, Length: 106, dtype: float64

In [10]:
y_test = pd.read_csv("dengue_features_test.csv")

In [11]:
y_test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,299.357143,295.308571,302.2,296.4,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,294.402857,303.0,296.9,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


#Simple Model 1: Average for each city

In [13]:
avg_total_cases_by_city = df.groupby("city")["total_cases"].mean()

In [16]:
avg_total_cases_by_city

city
iq     7.565385
sj    34.180556
Name: total_cases, dtype: float64

In [14]:
X_test = pd.read_csv("submission_format.csv")

In [15]:
X_test.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,0
1,sj,2008,19,0
2,sj,2008,20,0
3,sj,2008,21,0
4,sj,2008,22,0


In [19]:
X_output = X_test.assign(total_cases=X_test["city"].map(avg_total_cases_by_city))

In [20]:
X_output

,city,year,weekofyear,total_cases
0,sj,2008,18,34.180556
1,sj,2008,19,34.180556
2,sj,2008,20,34.180556
3,sj,2008,21,34.180556
4,sj,2008,22,34.180556
...,...,...,...,...
411,iq,2013,22,7.565385
412,iq,2013,23,7.565385
413,iq,2013,24,7.565385
414,iq,2013,25,7.565385


In [24]:
y_pred = X_train["city"].map(avg_total_cases_by_city)

In [22]:
from sklearn.metrics import r2_score

In [27]:
r2_score(y_train["total_cases"], y_pred)

0.08562883841184243

In [ ]:
#if i predict the mean of the training data
#then by R^2 score will be 0 for the training set

In [28]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_train["total_cases"], y_pred)

20.61348912839297

In [29]:
X_output.to_csv("mean_city_model.csv", index=False)

##Simple_2 model for each city and each week year

In [33]:
avg_city_week = df.groupby(["city", "weekofyear"])["total_cases"].mean()

In [43]:
y_pred = df[["city", "weekofyear"]].apply(lambda row: (row["city"], row["weekofyear"]), axis=1).map(avg_city_week)

In [44]:
r2_score(y_train["total_cases"], y_pred)

0.2319763643042393

In [45]:
mean_absolute_error(y_train["total_cases"], y_pred)

18.01633089133089

In [46]:
X_output = (
    X_test
    .assign(total_cases=lambda df: df[["city", "weekofyear"]]
    .apply(lambda row: (row["city"], row["weekofyear"]), axis=1).map(avg_city_week))
)

In [47]:
X_output

,city,year,weekofyear,total_cases
0,sj,2008,18,10.722222
1,sj,2008,19,9.944444
2,sj,2008,20,11.500000
3,sj,2008,21,11.166667
4,sj,2008,22,13.777778
...,...,...,...,...
411,iq,2013,22,4.800000
412,iq,2013,23,3.000000
413,iq,2013,24,3.000000
414,iq,2013,25,4.200000


In [48]:
X_output.to_csv("mean_city_week_model.csv", index=False)